In [ ]:
FG = nx.read_shp(os.path.join(location_graph, name_graph), 
                 simplify=True)
graph = graph_module.Graph()

lat_lon_to_index = {}
edge_id_counter = 0

HEIGHT_RESTRICED_NODES = {'77': 10}
# first node is closed, rest is locks which are closed on one side for now.
CLOSED_NODES = [143 ,365, 73, 208]


#New
MOVABLE_BRIDGES = [340, 103]

In [ ]:
class Lock():
    def __init__(self, args):
        self.nodes = args['nodes']
        self.operation_time = 0
        self.length = args['length']
        self.width = args['width']
        self.doors_open = args['doors_open']
        self.doors_close = args['doors_close']
        self.operating_time = args['operating_time']
        self.resource = args['resource']
        self.neighbour_lock = args['neighbour_lock']
        self.log = {"Message": [],
                    "Timestamp": [],
                    "Value": []
                   }
        self.lock_name = args["name"]

    def log_entry(self, log, t, value):
        """Log"""
        self.log["Message"].append(log)
        self.log["Timestamp"].append(t)
        self.log["Value"].append(value)

        
    def plot_arrivales(self):
        plothist = []
        time_delta = (self.log["Timestamp"][-1]-self.log["Timestamp"][0]) / 3600
        for i in range(len(self.log["Message"])):
            if "in" in self.log["Message"][i]:
                plothist.append(self.log["Timestamp"][i] / 3600)
        plt.figure()
        plt.ylabel('Vessels arriving at lock')
        plt.xlabel('time [h]')
        plt.hist(plothist, bins=int(time_delta))
        plt.show()

In [ ]:
# first entry of nodes corrsepondse to first entry neightbourlock
LOCKS = [
    Lock({'name':"Krammerlock",'nodes':[178,179], 'neighbour_lock':[317, 365],'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':1440, 'resource':simpy.PriorityResource(env, capacity = 3)}), 
    Lock({'name':"Hansweertlock",'nodes':[220,221], 'neighbour_lock':[208, 209], 'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':720, 'resource':simpy.PriorityResource(env, capacity = 3)}), 
    Lock({'name':"Kreekraklock",'nodes':[155, 156], 'neighbour_lock':[73,72], 'length':320,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':840, 'resource':simpy.PriorityResource(env, capacity = 3)}),
    Lock({'name':"Krammerlock",'nodes':[317, 365], 'neighbour_lock':[178,179],'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':1440, 'resource':simpy.PriorityResource(env, capacity = 3)}), 
    Lock({'name':"Hansweertlock",'nodes':[208, 209], 'neighbour_lock':[220,221], 'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':720, 'resource':simpy.PriorityResource(env, capacity = 3)}), 
    Lock({'name':"Kreekraklock",'nodes':[73,72], 'neighbour_lock':[155, 156], 'length':320,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':840, 'resource':simpy.PriorityResource(env, capacity = 3)})]

In [ ]:
# calculate distance between two points
def calculate_distance(orig, dest):
    wgs84 = pyproj.Geod(ellps='WGS84')
    distance = wgs84.inv(shapely.geometry.asShape(orig).x, shapely.geometry.asShape(orig).y, 
                         shapely.geometry.asShape(dest).x, shapely.geometry.asShape(dest).y)[2]
    return distance


def get_lock(node_one, node_two):
    for lock in LOCKS:
        nodes_lock = lock.nodes
        if node_one in nodes_lock and node_two in nodes_lock:
            return lock
    return None

#new
def get_bridge(index):
    if index in MOVABLE_BRIDGES:
        return True
    return False


def get_slow_down(index):
    if (get_bridge(index)):
        return True
    for lock in LOCKS:
        if index in lock.nodes:
            return True
    return False
    

def get_height_restriction(index):
    '''
    Restriction in meters for a node, vessel can only pass if height < restriction
    '''
    if (str(index) in HEIGHT_RESTRICED_NODES.keys()):
        print("found restriction {}".format(HEIGHT_RESTRICED_NODES[str(index)]))
        return HEIGHT_RESTRICED_NODES[str(index)]
    return 99

def get_closed(index):
    if index in CLOSED_NODES:
        return True
    return False

          
def do_nodes():
    for i, node in enumerate(FG.nodes(data = True)):
        graph.graph.add_node(i, pos = node[0], 
                        height_restriction = get_height_restriction(i),
                        closed = get_closed(i),
                        name = "Node {}".format(i),
                        bridge = get_bridge(i),
                        slow_down = get_slow_down(i),
                        geometry = shapely.geometry.Point(node[0][0], node[0][1]))

        lat_lon_to_index[node[0]] = i
        

def do_edges():
    for edge in FG.edges(data=True):
        edge_type = 'Canal'
        lock = get_lock(lat_lon_to_index[edge[0]],lat_lon_to_index[edge[1]])
        edge_attribute = None
        resource = simpy.Resource(env, capacity = math.inf)
        if (lock != None):
            edge_type = 'Lock'
            edge_attribute = lock
            resource = lock.resource
            print(lock)
        graph.graph.add_edge(lat_lon_to_index[edge[0]], 
                     lat_lon_to_index[edge[1]],
                     Object=edge_type,
                     dis= calculate_distance(nx.get_node_attributes(graph.graph, 'geometry')[lat_lon_to_index[edge[1]]], 
                                             nx.get_node_attributes(graph.graph, 'geometry')[lat_lon_to_index[edge[0]]]),
                     weight = 1,
                     Resources = resource,
                     attribute = edge_attribute
                    )

        
do_nodes()
do_edges()

FG = graph.graph
FG_S = graph.graph.to_directed()